# test environment

In [1]:
CONFIG_INI_DIR = '../../config.ini'

# config.ini
import configparser
config = configparser.ConfigParser()
config.read(CONFIG_INI_DIR)

# config.ini setting
log_folder = config['setting']['log_folder']
db_folder = config['setting']['db_folder']
db_name = config['setting']['hr_bank_db_name']

DB_CONNECTION_STRING = f'sqlite:///test_data/hr_bank_data_db.db'
DB_CONNECTION_STRING

'sqlite:///test_data/hr_bank_data_db.db'

In [2]:
import os

# logging
from logging.handlers import TimedRotatingFileHandler
# from scrapy.utils.log import configure_logging
import logging
log_filepath = os.path.join('/Users/yichen/Desktop/hr_bank/log', 'test_hr_bank_crawler.log')
logHandler = TimedRotatingFileHandler(log_filepath, when='midnight', interval=10)
# configure_logging(install_root_handler=False)
logging.basicConfig(handlers=[logHandler])

In [3]:
import sys
sys.path.append('..')

# import

In [4]:
import re
import json
import unicodedata
import pandas as pd
from sqlalchemy import and_
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError
from datetime import datetime
from urllib.parse import urljoin

from model.model import db_connect, create_table, TRawSearch, TRawJob, TRawJobAnalysis, TRawComp, TJob, TJobAnalysis, TComp


DOMAIN_URL = 'https://www.104.com.tw/'

# utils

In [5]:
def remove_unicode_blank(text):
    text = text.replace('\xa0', '')
    text = text.replace('\u3000', '')
    text = text.replace('\r', '')
    return text

    
def join_dict_item_in_list(l: list, key: str):
    ''' input:  [{"name": "Ian"}, {"name": "Wang"}]
        output: "Ian、Wang"
    '''
    dict_item_list = [d[key] for d in l]
    return '、'.join(dict_item_list)


def join_element_in_list(l: list) -> str:
    ''' input:  ["A", "B", "C"]
        output: "A、B、C"
    '''
    return '、'.join(l)


def clean_json(j: str) -> str:
    ''' convert full-width character to half-width one, such as "Ａ" to "A"
    '''
    j = unicodedata.normalize('NFKC', j)
    j = remove_unicode_blank(j)
    return j

# analysis

In [6]:
def _parse_analysis_content(crawled_dict: dict) -> dict:
    parsed_job = {}
    
    for type_, data in crawled_dict.items():
        
        # 資料更新時間
        if 'update_time' not in crawled_dict.keys():
            update_time = ''.join(re.findall('\d+', crawled_dict[type_]['update_time'][:10]))
            parsed_job['update_time'] = int(update_time)
        
        # 應徵人數
        if 'apply_count' not in crawled_dict.keys():
            parsed_job['apply_count'] = int(crawled_dict[type_]['total'])
    
    # 應徵人數資料整理
    apply_count_dict = _convert_analysis_json(crawled_dict)
    
    # 性別
    parsed_job['sex_m'] = apply_count_dict['sex']['男']
    parsed_job['sex_f'] = apply_count_dict['sex']['女']
    
    # 學歷
    parsed_job['edu_na'] = apply_count_dict['edu']['無法判斷']
    parsed_job['edu_no'] = apply_count_dict['edu']['不拘']
    parsed_job['edu_jr'] = apply_count_dict['edu']['國中(含)以下']
    parsed_job['edu_sr'] = apply_count_dict['edu']['高中職']
    parsed_job['edu_jr_coll'] = apply_count_dict['edu']['專科']
    parsed_job['edu_undergrad'] = apply_count_dict['edu']['大學']
    parsed_job['edu_grad'] = apply_count_dict['edu']['博碩士']

    # 年齡
    parsed_job['age_00_20'] = apply_count_dict['age']['20歲以下']
    parsed_job['age_21_25'] = apply_count_dict['age']['21~25歲']
    parsed_job['age_26_30'] = apply_count_dict['age']['26~30歲']
    parsed_job['age_31_35'] = apply_count_dict['age']['31~35歲']
    parsed_job['age_36_40'] = apply_count_dict['age']['36~40歲']
    parsed_job['age_41_45'] = apply_count_dict['age']['41~45歲']
    parsed_job['age_46_50'] = apply_count_dict['age']['46~50歲']
    parsed_job['age_51_55'] = apply_count_dict['age']['51~55歲']
    parsed_job['age_56_60'] = apply_count_dict['age']['56~60歲']
    parsed_job['age_60_99'] = apply_count_dict['age']['60歲以上']
    
    # 工作經驗
    parsed_job['work_exp_no'] = apply_count_dict['work_exp']['無工作經驗']
    parsed_job['work_exp_00_01'] = apply_count_dict['work_exp']['1年以下']
    parsed_job['work_exp_01_03'] = apply_count_dict['work_exp']['1~3年']
    parsed_job['work_exp_03_05'] = apply_count_dict['work_exp']['3~5年']
    parsed_job['work_exp_05_10'] = apply_count_dict['work_exp']['5~10年']
    parsed_job['work_exp_10_15'] = apply_count_dict['work_exp']['10~15年']
    parsed_job['work_exp_15_20'] = apply_count_dict['work_exp']['15~20年']
    parsed_job['work_exp_20_25'] = apply_count_dict['work_exp']['20~25年']
    parsed_job['work_exp_25_99'] = apply_count_dict['work_exp']['25年以上']

    # 語言
    filter_list = ['英文', '中文', '日文', '韓文']
    parsed_job['lang'] = apply_count_dict['lang'].get('英文', 0)
    parsed_job['lang'] = apply_count_dict['lang'].get('中文', 0)
    parsed_job['lang'] = apply_count_dict['lang'].get('日文', 0)
    parsed_job['lang'] = apply_count_dict['lang'].get('韓文', 0)
    parsed_job['lang'] = _get_other_apply_count(apply_count_dict['lang'], filter_list)
    
    # 科系
    filter_list = ['資訊管理相關', '資訊工程相關', '統計學相關', '數理統計相關']
    parsed_job['major_info_mgmt'] = apply_count_dict['major'].get('資訊管理相關', 0)
    parsed_job['major_cs'] = apply_count_dict['major'].get('資訊工程相關', 0)
    parsed_job['major_stat'] = apply_count_dict['major'].get('統計學相關', 0)
    parsed_job['major_math_stat'] = apply_count_dict['major'].get('數理統計相關', 0)
    parsed_job['major_other'] = _get_other_apply_count(apply_count_dict['major'], filter_list)

    # 技能
    filter_list = ['Java', 'Python']
    parsed_job['skill_java'] = apply_count_dict['skill'].get('Java', 0)
    parsed_job['skill_python'] = apply_count_dict['skill'].get('Python', 0)
    parsed_job['skill_other'] = _get_other_apply_count(apply_count_dict['skill'], filter_list)
    
    # 證照
    filter_list = ['國際專案管理師PMP']
    parsed_job['cert_pmp'] = apply_count_dict['cert'].get('國際專案管理師PMP', 0)
    parsed_job['cert_other'] = _get_other_apply_count(apply_count_dict['cert'], filter_list)

    return parsed_job
    

def _convert_analysis_json(analysis_dict: dict) -> dict:
    type_list = ['sex', 'edu', 'yearRange', 'exp', 'language', 'major', 'skill', 'cert']
    new_analysis_dict = {}

    for type_ in type_list:

        if type_ == 'sex':
            new_key = 'sex'

        elif type_ == 'edu':
            new_key = 'edu'

        elif type_ == 'yearRange':
            new_key = 'age'

        elif type_ == 'exp':
            new_key = 'work_exp'

        elif type_ == 'language':
            new_key = 'lang'

        elif type_ == 'major':
            new_key = 'major'

        elif type_ == 'skill':
            new_key = 'skill'

        elif type_ == 'cert':
            new_key = 'cert'
            
        else:
            new_key = type_

        new_analysis_dict[new_key] = {}
        for k1, v1 in analysis_dict[type_].items():
            if k1.isdigit():
                for k2, v2 in v1.items():
                    if 'Name' in k2:
                        new_analysis_dict[new_key][v2.strip()] = int(v1['count'])
                        break
                        
    return new_analysis_dict


def _get_other_apply_count(apply_count: dict, filter_list: list) -> int:
    other_count = 0
    for k, v in apply_count.items():
        if k in filter_list:
            continue
        other_count += v
    return other_count

# search

In [7]:
def _parse_job_list_content(crawled_dict: dict) -> dict:
    parsed_job = {}
    
    # 職缺類型
    parsed_job['job_type'] = crawled_dict['jobType']
    
    # 職缺編號
    parsed_job['job_no'] = crawled_dict['jobNo']
    
    # 職缺名稱
    parsed_job['job_name'] = crawled_dict['jobName']
    
    # 工作性質：全職、兼職...
    parsed_job['job_role'] = crawled_dict['jobRole']
    
    # 工作地區：台北市信義區...
    parsed_job['job_addr_dist'] = crawled_dict['jobAddrNoDesc']
    
    # 職缺內容
    parsed_job['job_detail'] = crawled_dict['description']
    
    # 學歷要求
    parsed_job['edu'] = crawled_dict['optionEdu']
    
    # 工作經驗要求
    parsed_job['work_exp'] = int(crawled_dict['period'])
    
    # 應徵人數
    parsed_job['apply_count'] = int(crawled_dict['applyCnt'])
    
    # 公司編號
    parsed_job['comp_no'] = crawled_dict['custNo']
    
    # 公司名稱
    parsed_job['comp_name'] = crawled_dict['custName']
    
    # 產業編號
    parsed_job['indust_no'] = crawled_dict['coIndustry']
    
    # 產業描述
    parsed_job['indust_desc'] = crawled_dict['coIndustryDesc']
    
    # 最低薪資
    parsed_job['salary_min'] = int(crawled_dict['salaryLow'])
    
    # 最高薪資
    parsed_job['salary_max'] = int(crawled_dict['salaryHigh'])
    
    # 薪資描述
    parsed_job['salary_desc'] = crawled_dict['salaryDesc']
    
    # 薪資類型
    parsed_job['salary_type'] = crawled_dict['s10']
    
    # 開缺日期
    parsed_job['appear_date'] = int(crawled_dict['appearDate'])
    
    # 職缺標籤
    parsed_job['job_tag'] = join_element_in_list(crawled_dict['tags'])
    
    # 工作地點地標
    parsed_job['landmark_tag'] = crawled_dict['landmark']
    
    # 職缺連結
    try:
        parsed_job['job_url'] = urljoin(DOMAIN_URL, crawled_dict['link']['job'])
    except:
        parsed_job['job_url'] = ''
    
    # 應徵分析連結
    try:
        parsed_job['analysis_url'] = urljoin(DOMAIN_URL, crawled_dict['link']['applyAnalyze'])
    except:
        parsed_job['analysis_url'] = ''
    
    # 公司連結
    try:
        parsed_job['comp_url'] = urljoin(DOMAIN_URL, crawled_dict['link']['cust'])
    except:
        parsed_job['comp_url'] = ''
    
    # 工作地點經度
    parsed_job['lon'] = crawled_dict['lon']
    
    # 工作地點緯度
    parsed_job['lat'] = crawled_dict['lat']
    
    return parsed_job


def _get_web_link(arg: str) -> str:
    try:
        return urljoin(DOMAIN_URL, crawled_dict['link'][arg])
    except:
        return ''

# detail

In [8]:
def _parse_job_detail_content(crawled_dict: dict) -> dict:
    parsed_job = {}
    
    # 職缺名稱
    parsed_job['job_name'] = crawled_dict['header']['jobName']
    
    # 開缺日期
#     parsed_job['appear_date'] = int(re.sub('/', '', crawled_dict['header']['appearDate'])) # second choice

    # 公司名稱
    parsed_job['comp_name'] = crawled_dict['header']['custName']
    
    # 公司連結
    try:
        parsed_job['comp_url'] = urljoin(DOMAIN_URL, crawled_dict['header']['custUrl'])
    except:
        parsed_job['comp_url'] = ''
    
    # 分析類型
    parsed_job['analysis_type'] = crawled_dict['header']['analysisType']
    
    # 分析連結
    try:
        parsed_job['analysis_url'] = urljoin(DOMAIN_URL, crawled_dict['header']['analysis_url'])
    except:
        parsed_job['analysis_url'] = ''
        
    # 接受身份
    parsed_job['accept_role'] = join_dict_item_in_list(
        crawled_dict['condition']['acceptRole']['role'], 'description') 
    
    # 婉拒身份
    parsed_job['accept_role'] = join_dict_item_in_list(
        crawled_dict['condition']['acceptRole']['disRole']['disability'], 'type') 
    
    # 工作經驗要求
#     parsed_job['work_exp'] = int(crawled_dict['condition']['workExp']) # second choice
    
    # 學歷要求
#     parsed_job['edu'] = crawled_dict['condition']['edu'] # second choice
    
    # 科系要求
    parsed_job['major'] = join_element_in_list(crawled_dict['condition']['major'])
    
    # 語言要求
    parsed_job['lang'] = join_dict_item_in_list(crawled_dict['condition']['language'], 'language')
    
    # 地方語言要求
    parsed_job['local_lang'] = join_dict_item_in_list(crawled_dict['condition']['localLanguage'], 'language')
    
    # 工具要求
    parsed_job['specialty'] = join_dict_item_in_list(crawled_dict['condition']['specialty'], 'description')
    
    # 技能要求
    parsed_job['skill'] = join_dict_item_in_list(crawled_dict['condition']['skill'], 'description')
    
    # 證照要求
    parsed_job['cert'] = join_element_in_list(crawled_dict['condition']['certificate'])
    
    # 駕照要求
#     parsed_job['driver_license'] = join_element_in_list(crawled_dict['condition']['driverLicense'])
    
    # 其他要求
    parsed_job['other'] = crawled_dict['condition']['other']
    
    # 職缺內容
    parsed_job['job_detail'] = crawled_dict['jobDetail']['jobDescription']
    
    # 職缺分類
    parsed_job['job_cat'] = join_dict_item_in_list(crawled_dict['jobDetail']['jobCategory'], 'description')
    
    # 最低薪資
    parsed_job['salary_min'] = int(crawled_dict['jobDetail']['salaryMin'])
    
    # 最高薪資
    parsed_job['salary_max'] = int(crawled_dict['jobDetail']['salaryMax'])
    
    # 薪資描述
    parsed_job['salary_desc'] = crawled_dict['jobDetail']['salary']
    
    # 薪資類型
    parsed_job['salary_type'] = crawled_dict['jobDetail']['salaryType']
    
    # 職缺類型
    parsed_job['job_type'] = crawled_dict['jobDetail']['jobType']
    
    # 工作性質：全職、兼職...
    parsed_job['job_role'] = crawled_dict['jobDetail']['workType'] # or work_type?
    
    # 工作地區：台北市信義區...
    parsed_job['job_addr_dist'] = crawled_dict['jobDetail']['addressRegion']
    
    # 工作地點經度
    parsed_job['lon'] = crawled_dict['jobDetail']['longitude']
    
    # 工作地點緯度
    parsed_job['lat'] = crawled_dict['jobDetail']['latitude']
    
    # 管理責任
    parsed_job['manage_resp'] = crawled_dict['jobDetail']['manageResp']
    
    # 出差外派
    parsed_job['business_trip'] = crawled_dict['jobDetail']['businessTrip']
    
    # 上班時段
    parsed_job['work_period'] = crawled_dict['jobDetail']['workPeriod']
    
    # 休假制度
    parsed_job['vacation_policy'] = crawled_dict['jobDetail']['vacationPolicy']
    
    # 可上班日
    parsed_job['start_work_day'] = crawled_dict['jobDetail']['startWorkingDay']
    
    # 招募類型：0公司自聘、1代徵
    parsed_job['hire_type'] = crawled_dict['jobDetail']['hireType']
    
    # 委託招募單位
    parsed_job['delegate_recruit'] = crawled_dict['jobDetail']['delegatedRecruit']
    
    # 需求人數
    parsed_job['need_emp'] = crawled_dict['jobDetail']['needEmp']
    
    # 最少需求人數
    parsed_job['need_count_min'] = _get_need_emp_min(parsed_job['need_emp'])
    
    # 最少需求人數
    parsed_job['need_count_max'] = _get_need_emp_max(parsed_job['need_emp'])
    
    # 職缺狀況
#     parsed_job['switch'] = crawled_dict['switch']['needEmp']
    
    # 產業描述
    parsed_job['indust_desc'] = crawled_dict['industry']
    
    # 公司編號
    parsed_job['comp_no'] = crawled_dict['custNo']
    
    return parsed_job
   
    
def _get_need_emp_min(need_emp_string):
    result = re.findall('(\d+)', need_emp_string)
    if result:
        return result[0]
    else:
        return 1
    
    
def _get_need_emp_max(need_emp_string):
    result = re.findall('(\d+)', need_emp_string)
    if result:
        return result[-1]
    else:
        return 99

# formatter

In [9]:
class JobDataFormatter:
    
    def __init__(self):
        engine = db_connect(DB_CONNECTION_STRING)
        create_table(engine)
        self.Session = sessionmaker(bind=engine)


    def process(self, crawl_date=None):
        session = self.Session()
        records = []
        
        try:
            if crawl_date:
                crawl_date = int(crawl_date)
                results = session.query(TRawWearch).filter(crawl_date==crawl_date).all()
            else:
                results = session.query(TRawSearch).all() # main table
        except Exception as e:
            print(e)
            logging.error('can not get data from "TRawSearch".')
            session.close()
            return records
        
        for i, r in enumerate(results):
            print(f'>>> processing {i} record...')
            job_no = r.job_no
            crawl_date = r.crawl_date
            logging.info(f'process record: JOB_NO={job_no}, CRAWL_DATE={crawl_date}')
            
            try:
                # job list content
                json_string = clean_json(r.json_string)
                search_json = json.loads(json_string)
                search_json = _parse_job_list_content(search_json)

                # job detail
                raw_job = session.query(TRawJob).filter(job_no==job_no, crawl_date==crawl_date).first()
                if raw_job:
                        json_string = clean_json(raw_job.json_string)
                        job_json = json.loads(json_string)
                        job_json = _parse_job_detail_content(job_json['data'])
                else:
                    job_json = {}

                # analysis
                raw_analysis = session.query(TRawJobAnalysis).filter(job_no==job_no, crawl_date==crawl_date).first()
                if raw_analysis:
                    json_string = clean_json(raw_analysis.json_string)
                    analysis_json = json.loads(json_string)
                    analysis_json = _parse_analysis_content(analysis_json)
                else:
                    analysis_json = {}
                
                # combine as one record
                record = {**analysis_json, **job_json, **search_json}
                records.append(record)
                print(f'>>> processing {i} record... Done!')
                
            except Exception as e:
                print(e)
                logging.error(f'No. {i} record:\nJOB_NO={job_no}\nCRAWL_DATE={crawl_date}\n{e}')
                
        session.close()
        return records

            
job_data_formatter = JobDataFormatter()
records = job_data_formatter.process()

>>> DB_CONNECTION_STRING: sqlite:///test_data/hr_bank_data_db.db
>>> processing 0 record...
>>> processing 0 record... Done!
>>> processing 1 record...
>>> processing 1 record... Done!
>>> processing 2 record...
>>> processing 2 record... Done!
>>> processing 3 record...
>>> processing 3 record... Done!
>>> processing 4 record...
>>> processing 4 record... Done!
>>> processing 5 record...
>>> processing 5 record... Done!
>>> processing 6 record...
>>> processing 6 record... Done!
>>> processing 7 record...
>>> processing 7 record... Done!
>>> processing 8 record...
>>> processing 8 record... Done!
>>> processing 9 record...
>>> processing 9 record... Done!
>>> processing 10 record...
>>> processing 10 record... Done!
>>> processing 11 record...
>>> processing 11 record... Done!
>>> processing 12 record...
>>> processing 12 record... Done!
>>> processing 13 record...
>>> processing 13 record... Done!
>>> processing 14 record...
>>> processing 14 record... Done!
>>> processing 15 record.

In [10]:
pd.set_option('display.max_columns', None)

In [23]:
map_column_name_to_desc_for_export = {
    
    'appear_date': '開缺日期',
    'update_time': '更新日期',
    'job_name': '職缺名稱',
    'job_no': '職缺編號',
    'job_url': '職缺連結',
    'comp_name': '公司名稱',
    'comp_no': '公司編號',
    'comp_url': '公司連結',
    'indust_desc': '產業描述',
    'indust_no': '產業編號',
    
    'need_emp': '需求人數',
    'need_count_max': '最多需求人數',
    'need_count_min': '最少需求人數',

    'job_cat': '工作分類',
    'job_detail': '職缺內容',
    'job_type': '職缺類型',
    'job_tag': '職缺標籤',
    'job_role': '工作性質',
    'salary_desc': '薪資描述',
    'salary_max': '最高薪資',
    'salary_min': '最低薪資',
    'salary_type': '薪資類型',
    'job_addr_dist': '工作地區',
    'manage_resp': '管理責任',
    'business_trip': '出差外派',
    'vacation_policy': '休假制度',
    
    'edu': '學歷要求',
    'major': '科系要求',
    'work_exp': '工作經驗要求',
    'skill': '技能要求',
    'specialty': '專長要求',
    'lang': '語言要求',
    'local_lang': '地方語言要求',
    'cert': '證照要求',
    'other': '其他要求',
    'accept_role': '接受身份',
    'start_work_day': '可上班日',
    
    'apply_count': '應徵人數',
    
    'analysis_type': '分析類型',
    'analysis_url': '分析連結',
    'age_00_20': '20歲以下',
    'age_21_25': '21-25歲',
    'age_26_30': '26-30歲',
    'age_31_35': '31-35歲',
    'age_36_40': '36-40歲',
    'age_41_45': '41-45歲',
    'age_46_50': '46-50歲',
    'age_51_55': '51-55歲',
    'age_56_60': '56-60歲',
    'age_60_99': '60歲以上',
    
    'major_cs': '資訊工程相關',
    'major_info_mgmt': '資訊管理相關',
    'major_math_stat': '數理統計相關',
    'major_stat': '統計學相關',
    'major_other': '其他科系',
    
    'work_exp_00_01': '1年以下',
    'work_exp_01_03': '1-3年',
    'work_exp_03_05': '3-5年',
    'work_exp_05_10': '5-10年',
    'work_exp_10_15': '10-15年',
    'work_exp_15_20': '15-20年',
    'work_exp_20_25': '20-25年',
    'work_exp_25_99': '25年以上',
    'work_exp_no': '無工作經驗',
    'work_period': '上班時段',
    
    'cert_other': '其他證照',
    'cert_pmp': 'PMP專案證照',
    
    'edu_jr': '國中',
    'edu_jr_coll': '專科',
    'edu_sr': '高中',
    'edu_undergrad': '大學',
    'edu_grad': '碩士以上',
    'edu_no': '學歷不拘',
    'edu_na': '無法判斷',
    
    'sex_f': '女性',
    'sex_m': '男性',
    
    'skill_java': 'Java',
    'skill_python': 'Python',
    'skill_other': '其他技能要求',

    'hire_type': '招募類型',
    'delegate_recruit': '託招募單位',
    
    'landmark_tag': '工作地點地標',
    'lat': '工作地點緯度',
    'lon': '工作地點經度',
}

In [28]:
df = pd.DataFrame(records)
df = df[list(map_column_name_to_desc_for_export.keys())]
new_column_name = list(map_column_name_to_desc_for_export.values())
df.columns = new_column_name
df.head(1)

,開缺日期,更新日期,職缺名稱,職缺編號,職缺連結,公司名稱,公司編號,公司連結,產業描述,產業編號,需求人數,最多需求人數,最少需求人數,工作分類,職缺內容,職缺類型,職缺標籤,工作性質,薪資描述,最高薪資,最低薪資,薪資類型,工作地區,管理責任,出差外派,休假制度,學歷要求,科系要求,工作經驗要求,技能要求,專長要求,語言要求,地方語言要求,證照要求,其他要求,接受身份,可上班日,應徵人數,分析類型,分析連結,20歲以下,21-25歲,26-30歲,31-35歲,36-40歲,41-45歲,46-50歲,51-55歲,56-60歲,60歲以上,資訊工程相關,資訊管理相關,數理統計相關,統計學相關,其他科系,1年以下,1-3年,3-5年,5-10年,10-15年,15-20年,20-25年,25年以上,無工作經驗,上班時段,其他證照,PMP專案證照,國中,專科,高中,大學,碩士以上,學歷不拘,無法判斷,女性,男性,Java,Python,其他技能要求,招募類型,託招募單位,工作地點地標,工作地點緯度,工作地點經度
0,20201225,20201227,【RD】資安開發工程師- 2000781,12008786,https://www.104.com.tw/job/75e1e?jobsource=n10...,安碁資訊股份有限公司,70565450000,https://www.104.com.tw/company/wf0uhow?jobsour...,網際網路相關業,1001001003,1人,1,1,產品企劃開發人員、MIS程式設計師、Internet程式設計師,1. 資安產品開發\n2. 資安服務自動化流程規劃\n3. 資安弱點研究與漏洞挖掘\n,0,員工230人,1,"月薪35,000元以上",9999999,35000,50,台北市信義區,不需負擔管理責任,無需出差外派,週休二日,大學,資訊管理相關,3,,,,,,"1. 大學(含)以上畢業, 至少2年(含)以上工作經驗\n2. 優秀的學習能力,分析和解決問...",,一個月內,0,1,https://www.104.com.tw/jobs/apply/analysis/75e...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,日班,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,距捷運永春站410公尺,25.0409201,121.5720055


In [ ]:
result = session.query(TRawSearch).all()
for i, r in enumerate(result):
    print(i)
    job_no = r.job_no
    crawl_date = r.crawl_date
    
    # job list content
    json_string = clean_json(r.json_string)
    search_json = json.loads(json_string)
    search_json = _parse_job_list_content(search_json)
    
    # job detail
    raw_job = session.query(TRawJob).filter(job_no==job_no, crawl_date==crawl_date).first()
    json_string = clean_json(raw_job.json_string)
    job_json = json.loads(json_string)
    job_json = _parse_job_detail_content(job_json['data'])
    
    # analysis
    raw_analysis = session.query(TRawJobAnalysis).filter(job_no==job_no, crawl_date==crawl_date).first()
    json_string = clean_json(raw_analysis.json_string)
    analysis_json = json.loads(json_string)
    analysis_json = _parse_analysis_content(analysis_json)
    
    print({**analysis_json, **job_json, **search_json})
    print('\n\n')
#     break

In [ ]:

json_string = unicodedata.normalize('NFKC', r.json_string)
json_string = remove_unicode_blank(json_string)
analysis_dict = json.loads(json_string)
for k in analysis_dict.keys():
    print(f"'{k}': '',")

In [ ]:
analysis_dict

In [ ]:
map_search_json_key = {
    'jobType': 'job_type',
    'jobNo': 'job_no',
    'jobName': 'job_name',
    'jobRole': 'job_role',
    'jobAddrNoDesc': 'job_addr_dist',
    'description': 'job_detail',
    'optionEdu': 'edu',
    'period': 'work_exp',
    'applyCnt': 'apply_count',
    'custNo': 'comp_no',
    'custName': 'comp_name',
    'coIndustry': 'indust_no',
    'coIndustryDesc': 'indust_desc',
    'salaryLow': 'salary_min',
    'salaryHigh': 'salary_max',
    'salaryDesc': 'salary_desc',
    's10': 'salary_type',
    'appearDate': 'appear_date',
    'tags': 'job_tag',
    'landmark': 'landmark_tag',
    'link': 'links',
    'lon': 'lon',
    'lat': 'lat',
}

map_job_json_key = {
    'jobName': 'job_name',
    'appearDate': 'appear_date',
    'custName': 'comp_name',
    'custUrl': 'comp_url',
    'analysisType': 'analysis_type',
    'analysisUrl': 'analysis_url',
    'acceptRole': 'accept_role',
    'disRole': 'disaccept_role',
    'workExp': 'work_exp',
    'edu': 'edu',
    'major': 'major',
    'language': 'lang',
    'localLanguage': 'local_lang',
    'specialty': 'specialty',
    'skill': 'skill',
    'certificate': 'cert',
    'driverLicense': 'driver_license',
    'other': 'other',
    'legalTag': 'legal_tag',
    'jobDescription': 'job_detail',
    'jobCategory': 'job_cat',
    'salary': 'salary_desc',
    'salaryMin': 'salary_min',
    'salaryMax': 'salary_max',
    'salaryType': 'salary_type',
    'jobType': 'job_type',
    'workType': 'work_type',
    'addressRegion': 'job_addr_dist',
    'longitude': 'lon',
    'latitude': 'lat',
    'manageResp': 'manage_resp',
    'businessTrip': 'business_trip',
    'workPeriod': 'work_period',
    'vacationPolicy': 'vacation_policy',
    'startWorkingDay': 'start_work_day',
    'hireType': 'hire_type',
    'delegatedRecruit': 'delegate_recruit',
    'needEmp': 'need_emp',
    'switch': 'switch',
    'industry': 'indust_desc',
    'custNo': 'comp_no',
}

In [ ]:
def _parse_analysis_content(crawled_dict: dict) -> dict:
    parsed_job = {}
    
    for type_, data in crawled_dict.items():
        
        # 資料更新時間
        if 'update_time' not in crawled_dict.keys():
            update_time = ''.join(re.findall('\d+', crawled_dict[type_]['update_time'][:10]))
            parsed_job['update_time'] = int(update_time)
        
        # 應徵人數
        if 'apply_count' not in crawled_dict.keys():
            parsed_job['apply_count'] = int(crawled_dict[type_]['total'])
    
    # 應徵人數資料整理
    apply_count_dict = _convert_analysis_json(crawled_dict)
    
    # 性別
    parsed_job['sex_m'] = apply_count_dict['sex']['男']
    parsed_job['sex_f'] = apply_count_dict['sex']['女']
    
    # 學歷
    parsed_job['edu_na'] = apply_count_dict['edu']['無法判斷']
    parsed_job['edu_no'] = apply_count_dict['edu']['不拘']
    parsed_job['edu_jr'] = apply_count_dict['edu']['國中(含)以下']
    parsed_job['edu_sr'] = apply_count_dict['edu']['高中職']
    parsed_job['edu_jr_coll'] = apply_count_dict['edu']['專科']
    parsed_job['edu_undergrad'] = apply_count_dict['edu']['大學']
    parsed_job['edu_grad'] = apply_count_dict['edu']['博碩士']

    # 年齡
    parsed_job['age_00_20'] = apply_count_dict['age']['20歲以下']
    parsed_job['age_21_25'] = apply_count_dict['age']['21~25歲']
    parsed_job['age_26_30'] = apply_count_dict['age']['26~30歲']
    parsed_job['age_31_35'] = apply_count_dict['age']['31~35歲']
    parsed_job['age_36_40'] = apply_count_dict['age']['36~40歲']
    parsed_job['age_41_45'] = apply_count_dict['age']['41~45歲']
    parsed_job['age_46_50'] = apply_count_dict['age']['46~50歲']
    parsed_job['age_51_55'] = apply_count_dict['age']['51~55歲']
    parsed_job['age_56_60'] = apply_count_dict['age']['56~60歲']
    parsed_job['age_60_99'] = apply_count_dict['age']['60歲以上']
    
    # 工作經驗
    parsed_job['work_exp_no'] = apply_count_dict['work_exp']['無工作經驗']
    parsed_job['work_exp_00_01'] = apply_count_dict['work_exp']['1年以下']
    parsed_job['work_exp_01_03'] = apply_count_dict['work_exp']['1~3年']
    parsed_job['work_exp_03_05'] = apply_count_dict['work_exp']['3~5年']
    parsed_job['work_exp_05_10'] = apply_count_dict['work_exp']['5~10年']
    parsed_job['work_exp_10_15'] = apply_count_dict['work_exp']['10~15年']
    parsed_job['work_exp_15_20'] = apply_count_dict['work_exp']['15~20年']
    parsed_job['work_exp_20_25'] = apply_count_dict['work_exp']['20~25年']
    parsed_job['work_exp_25_99'] = apply_count_dict['work_exp']['25年以上']

    # 語言
    filter_list = ['英文', '中文', '日文', '韓文']
    parsed_job['lang'] = apply_count_dict['lang'].get('英文', 0)
    parsed_job['lang'] = apply_count_dict['lang'].get('中文', 0)
    parsed_job['lang'] = apply_count_dict['lang'].get('日文', 0)
    parsed_job['lang'] = apply_count_dict['lang'].get('韓文', 0)
    parsed_job['lang'] = _get_other_apply_count(apply_count_dict['lang'], filter_list)
    
    # 科系
    filter_list = ['資訊管理相關', '資訊工程相關', '統計學相關', '數理統計相關']
    parsed_job['major_info_mgmt'] = apply_count_dict['major'].get('資訊管理相關', 0)
    parsed_job['major_cs'] = apply_count_dict['major'].get('資訊工程相關', 0)
    parsed_job['major_stat'] = apply_count_dict['major'].get('統計學相關', 0)
    parsed_job['major_math_stat'] = apply_count_dict['major'].get('數理統計相關', 0)
    parsed_job['major_other'] = _get_other_apply_count(apply_count_dict['major'], filter_list)

    # 技能
    filter_list = ['Java', 'Python']
    parsed_job['skill_java'] = apply_count_dict['skill'].get('Java', 0)
    parsed_job['skill_python'] = apply_count_dict['skill'].get('Python', 0)
    parsed_job['skill_other'] = _get_other_apply_count(apply_count_dict['skill'], filter_list)
    
    # 證照
    filter_list = ['國際專案管理師PMP']
    parsed_job['cert_pmp'] = apply_count_dict['cert'].get('國際專案管理師PMP', 0)
    parsed_job['cert_other'] = _get_other_apply_count(apply_count_dict['cert'], filter_list)

    return parsed_job
    

def _convert_analysis_json(analysis_dict: dict) -> dict:
    type_list = ['sex', 'edu', 'yearRange', 'exp', 'language', 'major', 'skill', 'cert']
    new_analysis_dict = {}

    for type_ in type_list:

        if type_ == 'sex':
            new_key = 'sex'

        elif type_ == 'edu':
            new_key = 'edu'

        elif type_ == 'yearRange':
            new_key = 'age'

        elif type_ == 'exp':
            new_key = 'work_exp'

        elif type_ == 'language':
            new_key = 'lang'

        elif type_ == 'major':
            new_key = 'major'

        elif type_ == 'skill':
            new_key = 'skill'

        elif type_ == 'cert':
            new_key = 'cert'
            
        else:
            new_key = type_

        new_analysis_dict[new_key] = {}
        for k1, v1 in analysis_dict[type_].items():
            if k1.isdigit():
                for k2, v2 in v1.items():
                    if 'Name' in k2:
                        new_analysis_dict[new_key][v2.strip()] = int(v1['count'])
                        break
                        
    return new_analysis_dict


def _get_other_apply_count(apply_count: dict, filter_list: list) -> int:
    other_count = 0
    for k, v in apply_count.items():
        if k in filter_list:
            continue
        other_count += v
    return other_count



_parse_analysis_content(analysis_dict)

In [ ]:
def _parse_job_detail_content(crawled_dict: dict) -> dict:
    parsed_job = {}
    
    # 職缺名稱
    parsed_job['job_name'] = crawled_dict['header']['jobName']
    
    # 開缺日期
#     parsed_job['appear_date'] = int(re.sub('/', '', crawled_dict['header']['appearDate'])) # second choice

    # 公司名稱
    parsed_job['comp_name'] = crawled_dict['header']['custName']
    
    # 公司連結
    try:
        parsed_job['comp_url'] = urljoin(DOMAIN_URL, crawled_dict['header']['custUrl'])
    except:
        parsed_job['comp_url'] = ''
    
    # 分析類型
    parsed_job['analysis_type'] = crawled_dict['header']['analysisType']
    
    # 分析連結
    try:
        parsed_job['analysis_url'] = urljoin(DOMAIN_URL, crawled_dict['header']['analysis_url'])
    except:
        parsed_job['analysis_url'] = ''
        
    # 接受身份
    parsed_job['accept_role'] = join_dict_item_in_list(
        crawled_dict['condition']['acceptRole']['role'], 'description') 
    
    # 婉拒身份
    parsed_job['accept_role'] = join_dict_item_in_list(
        crawled_dict['condition']['acceptRole']['disRole']['disability'], 'type') 
    
    # 工作經驗要求
#     parsed_job['work_exp'] = int(crawled_dict['condition']['workExp']) # second choice
    
    # 學歷要求
#     parsed_job['edu'] = crawled_dict['condition']['edu'] # second choice
    
    # 科系要求
    parsed_job['major'] = join_element_in_list(crawled_dict['condition']['major'])
    
    # 語言要求
    parsed_job['lang'] = join_dict_item_in_list(crawled_dict['condition']['language'], 'language')
    
    # 地方語言要求
    parsed_job['local_lang'] = join_dict_item_in_list(crawled_dict['condition']['localLanguage'], 'language')
    
    # 工具要求
    parsed_job['specialty'] = join_dict_item_in_list(crawled_dict['condition']['specialty'], 'description')
    
    # 技能要求
    parsed_job['skill'] = join_dict_item_in_list(crawled_dict['condition']['skill'], 'description')
    
    # 證照要求
    parsed_job['cert'] = join_element_in_list(crawled_dict['condition']['certificate'])
    
    # 駕照要求
#     parsed_job['driver_license'] = join_element_in_list(crawled_dict['condition']['driverLicense'])
    
    # 其他要求
    parsed_job['other'] = crawled_dict['condition']['other']
    
    # 職缺內容
    parsed_job['job_detail'] = crawled_dict['jobDetail']['jobDescription']
    
    # 職缺分類
    parsed_job['job_cat'] = join_dict_item_in_list(crawled_dict['jobDetail']['jobCategory'], 'description')
    
    # 最低薪資
    parsed_job['salary_min'] = int(crawled_dict['jobDetail']['salaryMin'])
    
    # 最高薪資
    parsed_job['salary_max'] = int(crawled_dict['jobDetail']['salaryMax'])
    
    # 薪資描述
    parsed_job['salary_desc'] = crawled_dict['jobDetail']['salary']
    
    # 薪資類型
    parsed_job['salary_type'] = crawled_dict['jobDetail']['salaryType']
    
    # 職缺類型
    parsed_job['job_type'] = crawled_dict['jobDetail']['jobType']
    
    # 工作性質：全職、兼職...
    parsed_job['job_role'] = crawled_dict['jobDetail']['workType'] # or work_type?
    
    # 工作地區：台北市信義區...
    parsed_job['job_addr_dist'] = crawled_dict['jobDetail']['addressRegion']
    
    # 工作地點經度
    parsed_job['lon'] = crawled_dict['jobDetail']['longitude']
    
    # 工作地點緯度
    parsed_job['lat'] = crawled_dict['jobDetail']['latitude']
    
    # 管理責任
    parsed_job['manage_resp'] = crawled_dict['jobDetail']['manageResp']
    
    # 出差外派
    parsed_job['business_trip'] = crawled_dict['jobDetail']['businessTrip']
    
    # 上班時段
    parsed_job['work_period'] = crawled_dict['jobDetail']['workPeriod']
    
    # 休假制度
    parsed_job['vacation_policy'] = crawled_dict['jobDetail']['vacationPolicy']
    
    # 可上班日
    parsed_job['start_work_day'] = crawled_dict['jobDetail']['startWorkingDay']
    
    # 招募類型：0公司自聘、1代徵
    parsed_job['hire_type'] = crawled_dict['jobDetail']['hireType']
    
    # 委託招募單位
    parsed_job['delegate_recruit'] = crawled_dict['jobDetail']['delegatedRecruit']
    
    # 需求人數
    parsed_job['need_emp'] = crawled_dict['jobDetail']['needEmp']
    
    # 最少需求人數
    parsed_job['need_count_min'] = _get_need_emp_min(parsed_job['need_emp'])
    
    # 最多需求人數
    parsed_job['need_count_max'] = _get_need_emp_max(parsed_job['need_emp'])
    
    # 職缺狀況
#     parsed_job['switch'] = crawled_dict['switch']['needEmp']
    
    # 產業描述
    parsed_job['indust_desc'] = crawled_dict['industry']
    
    # 公司編號
    parsed_job['comp_no'] = crawled_dict['custNo']
    
    return parsed_job
   
    
def _get_need_emp_min(need_emp_string):
    result = re.findall('(\d+)', need_emp_string)
    if result:
        return result[0]
    else:
        return 1
    
    
def _get_need_emp_max(need_emp_string):
    result = re.findall('(\d+)', need_emp_string)
    if result:
        return result[-1]
    else:
        return 99
    
_parse_job_detail_content(job_dict['data'])  

In [ ]:
DOMAIN_URL = 'https://www.104.com.tw/'

def _parse_job_list_content(crawled_dict: dict) -> dict:
    parsed_job = {}
    
    # 職缺類型
    parsed_job['job_type'] = crawled_dict['jobType']
    
    # 職缺編號
    parsed_job['job_no'] = crawled_dict['jobNo']
    
    # 職缺名稱
    parsed_job['job_name'] = crawled_dict['jobName']
    
    # 工作性質：全職、兼職...
    parsed_job['job_role'] = crawled_dict['jobRole']
    
    # 工作地區：台北市信義區...
    parsed_job['job_addr_dist'] = crawled_dict['jobAddrNoDesc']
    
    # 職缺內容
    parsed_job['job_detail'] = crawled_dict['description']
    
    # 學歷要求
    parsed_job['edu'] = crawled_dict['optionEdu']
    
    # 工作經驗要求
    parsed_job['work_exp'] = int(crawled_dict['period'])
    
    # 應徵人數
    parsed_job['apply_count'] = int(crawled_dict['applyCnt'])
    
    # 公司編號
    parsed_job['comp_no'] = crawled_dict['custNo']
    
    # 公司名稱
    parsed_job['comp_name'] = crawled_dict['custName']
    
    # 產業編號
    parsed_job['indust_no'] = crawled_dict['coIndustry']
    
    # 產業描述
    parsed_job['indust_desc'] = crawled_dict['coIndustryDesc']
    
    # 最低薪資
    parsed_job['salary_min'] = int(crawled_dict['salaryLow'])
    
    # 最高薪資
    parsed_job['salary_max'] = int(crawled_dict['salaryHigh'])
    
    # 薪資描述
    parsed_job['salary_desc'] = crawled_dict['salaryDesc']
    
    # 薪資類型
    parsed_job['salary_type'] = crawled_dict['s10']
    
    # 開缺日期
    parsed_job['appear_date'] = int(crawled_dict['appearDate'])
    
    # 職缺標籤
    parsed_job['job_tag'] = crawled_dict['tags']
    
    # 工作地點地標
    parsed_job['landmark_tag'] = crawled_dict['landmark']
    
    # 職缺連結
    try:
        parsed_job['job_url'] = urljoin(DOMAIN_URL, crawled_dict['link']['job'])
    except:
        parsed_job['job_url'] = ''
    
    # 應徵分析連結
    try:
        parsed_job['analysis_url'] = urljoin(DOMAIN_URL, crawled_dict['link']['applyAnalyze'])
    except:
        parsed_job['analysis_url'] = ''
    
    # 公司連結
    try:
        parsed_job['comp_url'] = urljoin(DOMAIN_URL, crawled_dict['link']['cust'])
    except:
        parsed_job['comp_url'] = ''
    
    # 工作地點經度
    parsed_job['lon'] = crawled_dict['lon']
    
    # 工作地點緯度
    parsed_job['lat'] = crawled_dict['lat']
    
    return parsed_job


def _get_web_link(arg: str) -> str:
    try:
        return urljoin(DOMAIN_URL, crawled_dict['link'][arg])
    except:
        return ''
    
    
_parse_job_list_content(search_dict)

In [ ]:
d = {}
for k, v in search_dict.items():
    if k in map_search_json_key.keys():
        new_key = map_search_json_key[k]
        d[new_key] = v
d

In [ ]:
def flat_dict(d, return_dict={}):
    for key, value in d.items():
        if isinstance(value, dict):
            return_dict = get_flat_dict(value, return_dict)
        else:
            return_dict[key] = value
    return return_dict